In [23]:
params = {'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'3h'
         ,'historical_retention':'7d'
         ,'real_time_table_name':'faker'
         ,'config_path':'/User/parquez/config/parquez.ini'}

In [24]:
def create_cron_string(params):
    import re
    PARTITION_BY_RE = r"([0-9]+)([a-zA-Z]+)"
    m = re.match(PARTITION_BY_RE, params.get("partition_interval"))
    if m.group(2) == 'm':
        result = "*/" + m.group(1) + " * * * * "
    if m.group(2) == 'h':
        result = "0 " + "*/" + m.group(1) + " * * * "
    if m.group(2) == 'd':
        if m.group(1) == 1:
            result = "0 0 " + "* * * "
        else:
            result = "0 0 " + "*/" + m.group(1) + " * * "
    if m.group(2) == 'M':
        result = "0 0 0" + "*/" + m.group(1) + " * "
    if m.group(2) == 'DW':
        result = "0 0 0 0 " + "*/" + m.group(1)
    return result

In [25]:
cron_string = create_cron_string(params) 
print(cron_string)

0 */1 * * * 


In [26]:
from os import path, getenv
from mlrun import new_project, mlconf

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "parquez"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/parquez
Project name: parquez


In [27]:
out = mlconf.artifact_path or path.abspath('./data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')

In [28]:
%env PYTHONPATH=./

env: PYTHONPATH=./


In [29]:
project.set_function("run_parquez_interval.py", 'crontab', kind='job', image='aviaigz/parquez')

In [30]:
from mlrun import run_local, mount_v3io

In [31]:
project.func('crontab').apply(mount_v3io())
project.func('crontab').set_env('PYTHONPATH', project_path)
project.func('crontab').spec.artifact_path = 'User/artifacts'
project.func('crontab').spec.service_account='mlrun-api'

In [ ]:
artifact_path = '/User/artifacts'
# #project.func('parquezrun').run()
project.func('crontab').run( params = {'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'3h'
         ,'historical_retention':'7d'
         ,'real_time_table_name':'faker'
         ,'config_path':'/User/parquez/config/parquez.ini'},artifact_path=artifact_path,)

[mlrun] 2020-07-21 17:07:33,263 starting run crontab uid=9a9b18499d91409e94d8ec232dcb0eab  -> http://mlrun-api:8080
[mlrun] 2020-07-21 17:07:33,594 Job is running in the background, pod: crontab-ztbvs
[mlrun] 2020-07-21 17:07:41,536 warning!, server (0.4.10) and client (0.4.9) ver dont match
[mlrun] 2020-07-21 17:07:41,581 starting local run: main.py # main
